### Import des bibliothéques
---

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

###  Chargement des données
---

In [2]:
df = pd.read_csv("data/get_around_delay_analysis.csv", encoding="latin-1")
df.head()



,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


### L'EDA Analyse exploratoire des données Basique
---

-rental_id' → 'identifiant_location'<br>
-car_id' → 'identifiant_voiture'<br>
-checkin_type' → 'type_enregistrement'<br>
-state' → 'état'<br>
-delay_at_checkout_in_minutes' → 'délai_au_checkout_en_minutes'<br>
-previous_ended_rental_id' → 'identifiant_location_terminée_précédente'<br>
-time_delta_with_previous_rental_in_minutes' → 'délai_avec_location_précédente_en_minutes'<br>

In [3]:
print(f"Nous avons {df.shape[1]} colonnes et {df.shape[0]} lignes dans ce jeu de données")

print("\n... Informations sur les colonnes et les types de données ...")
df.info() 

print("\n... Statistiques descriptives sur les colonnes numériques ...")
display(df.describe())

print("\n... Valeurs manquantes ...")
nb_valeurs_manquantes = df.isnull().sum()
pourcentage_valeurs_manquantes = (df.isnull().mean() * 100).round(2)
df_manquants = pd.DataFrame({
    'Nombre de Valeurs Manquantes': nb_valeurs_manquantes,
    '% Valeurs Manquantes': pourcentage_valeurs_manquantes
})
# Ne garder que les colonnes avec des valeurs manquantes
df_manquants[df_manquants['Nombre de Valeurs Manquantes'] > 0].sort_values('Nombre de Valeurs Manquantes', ascending=False)
print(df_manquants)

Nous avons 7 colonnes et 21310 lignes dans ce jeu de données

... Informations sur les colonnes et les types de données ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.1+ MB

... Statistiques descriptives sur les colonnes n

,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,16346.000000,1841.000000,1841.000000
mean,549712.880338,350030.603426,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,67.000000,560823.000000,540.000000
max,576401.000000,417675.000000,71084.000000,575053.000000,720.000000



... Valeurs manquantes ...
                                            Nombre de Valeurs Manquantes  \
rental_id                                                              0   
car_id                                                                 0   
checkin_type                                                           0   
state                                                                  0   
delay_at_checkout_in_minutes                                        4964   
previous_ended_rental_id                                           19469   
time_delta_with_previous_rental_in_minutes                         19469   

                                            % Valeurs Manquantes  
rental_id                                                   0.00  
car_id                                                      0.00  
checkin_type                                                0.00  
state                                                       0.00  
delay_at_checkout_in_minutes

### L'EDA Analyse exploratoire des données Approfondie
---

In [4]:
# Analyse col 'checkin_type'
print('... Analyse chekin_type ...\n Pratiquement 80% des locations se font sur mobile')
type=df['checkin_type'].value_counts().reset_index()
type.columns=['Type','Count']

# visualistaion chekin_type     
fig = px.pie(type, values='Count',names='Type', title='Répartition des types de check-in')
fig.update_traces(textposition='inside', textinfo='label+percent')
fig.show()

# Analyse col 'state'
print('... Analyse state ...\n')
df['state'].value_counts()

# State 'canceled'
df_canceled = df[df['state'] == 'canceled']
df_canceled.describe(include='all') 

print("Les locations avec un statut 'canceled' indiquent que la location n'a pas eu lieu.")
# State 'ended'
ended = df[df['state'] == 'ended'].copy()
print(f"\nNous avons {ended.shape[1]} colonnes et {ended.shape[0]} lignes dpour les locations ended\n")

# Analyse col 'delay_at_checkout_in_minutes'
delay_at_checkout = ended['delay_at_checkout_in_minutes'].describe()
print(f" ....Analyse des retards ....\n{delay_at_checkout}")
      
print("\nEn moyenne les voitures sont rendues avec 61 minutes de retard. \n" \
"L'écart type est énorme 993(16h). Valeur min et max sont extrémes. \n" \
"Plus de 50% des locations sont rendues avec un retard de moins de 10 min.")

# Retards Réells
print("\n... Les retards réels ...\n")
delay_real = ended[ended["delay_at_checkout_in_minutes"] > 0]
delay_real.info()

# Valeurs extrêmes
q95 = delay_real["delay_at_checkout_in_minutes"].quantile(0.95)
filtered = delay_real[delay_real["delay_at_checkout_in_minutes"] <= q95]

mean_delay = filtered["delay_at_checkout_in_minutes"].mean()
median_delay = filtered["delay_at_checkout_in_minutes"].median()
std_delay = filtered["delay_at_checkout_in_minutes"].std()

print(f"\nLa moy des retards est de {mean_delay:.1f} minutes. \nLa médiane se situe à {median_delay:.1f} minutes. \nL'écart-type de {std_delay:.1f} minutes montre certains condusteurs dépassent largement l'heure.\n")

# Visualisation des retards réells
fig = px.histogram(
    filtered,
    x="delay_at_checkout_in_minutes",
    nbins=50,
    title="Distribution des retards réels",
    labels={"delay_at_checkout_in_minutes": "Retard en minutes"}
)
fig.add_vline(x=filtered["delay_at_checkout_in_minutes"].mean(), line_color="red",annotation_text="Mean",
    annotation_position="top")
fig.add_vline(x=filtered["delay_at_checkout_in_minutes"].median(), line_color="green", annotation_text="Median",
    annotation_position="top")

fig.show()

# Analyse col 'time_delta_with_previous_rental_in_minutes'(délais >< 2 loc de la même voiture)
delay_2_loc= ended['time_delta_with_previous_rental_in_minutes'].describe()
print(f" ....Analyse du délais entre 2 locations d'une même voiture ....\n{delay_2_loc}\n")
      
print("\nEn moyenne le délais entre 2 locations est de 277 minutes, plus de 4h30. \nL'écart type est élévé 255 minutes, forte variabilité (certains vehicules sont reloués en - de 1h et d'autres 12h)\n50% des locations ont un delta entre 2 locations de 3h.")



... Analyse chekin_type ...
 Pratiquement 80% des locations se font sur mobile


... Analyse state ...

Les locations avec un statut 'canceled' indiquent que la location n'a pas eu lieu.

Nous avons 7 colonnes et 18045 lignes dpour les locations ended

 ....Analyse des retards ....
count    16345.000000
mean        60.773876
std        993.173222
min     -22433.000000
25%        -36.000000
50%          9.000000
75%         67.000000
max      71084.000000
Name: delay_at_checkout_in_minutes, dtype: float64

En moyenne les voitures sont rendues avec 61 minutes de retard. 
L'écart type est énorme 993(16h). Valeur min et max sont extrémes. 
Plus de 50% des locations sont rendues avec un retard de moins de 10 min.

... Les retards réels ...

<class 'pandas.core.frame.DataFrame'>
Index: 9404 entries, 2 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   9404 non-null   int64  
 1   car_id   

 ....Analyse du délais entre 2 locations d'une même voiture ....
count    1612.000000
mean      277.071960
std       255.157331
min         0.000000
25%        60.000000
50%       180.000000
75%       540.000000
max       720.000000
Name: time_delta_with_previous_rental_in_minutes, dtype: float64


En moyenne le délais entre 2 locations est de 277 minutes, plus de 4h30. 
L'écart type est élévé 255 minutes, forte variabilité (certains vehicules sont reloués en - de 1h et d'autres 12h)
50% des locations ont un delta entre 2 locations de 3h.


### Analyse du seuil : quelle doit être la durée minimale du délai ?
---

In [6]:
# Calcul de la durée minimale obligatoire entre 2 locations : seuil 30 min
print("\n... Seuil entre 2 loctaions ...\n")
seuil = 30 
nbr_loc_bloquee = (ended['time_delta_with_previous_rental_in_minutes']<seuil).sum()
nbr_loc_ok = (ended['time_delta_with_previous_rental_in_minutes']>=seuil).sum()
print(f"Pour un seuil de 30 minutes")
print(f" {nbr_loc_bloquee} locations seront bloquées et {nbr_loc_ok} loctaions seront possibles")

# test sur plusieurs seuils
seuil=[30, 60, 90, 180, 240]
resultat=[]
nbr_car_delay_2_loc = len(ended.dropna(subset=['time_delta_with_previous_rental_in_minutes']))

for i in seuil:
    nbr_loc_bloquee = (ended['time_delta_with_previous_rental_in_minutes']<i).sum()
    pct_nbr_loc_bloquee = ((nbr_loc_bloquee / nbr_car_delay_2_loc) * 100).round(2)
    nbr_loc_ok = (ended['time_delta_with_previous_rental_in_minutes']>=i).sum()
    pct_nbr_loc_ok = ((nbr_loc_ok / nbr_car_delay_2_loc) * 100).round(2)
 
    # Retards réels couverts (seulement pour les retards > 0)
    retards_reels = ended[ended['delay_at_checkout_in_minutes'] > 0]
    pct_retards_couverts = ((retards_reels['delay_at_checkout_in_minutes'] <= i).mean() * 100).round(2)

    # Ajouter les résultats à la liste
    resultat.append({
        'Seuil (min)': i,
        'Locations bloquées': nbr_loc_bloquee,
        '% Loc bloquées': pct_nbr_loc_bloquee,
        'Locations possibles' : nbr_loc_ok,
        '% Loc possibles': pct_nbr_loc_ok,
        '% Retards réels couverts': pct_retards_couverts,
    })
df_resultat = pd.DataFrame(resultat)
df_resultat.head()





... Seuil entre 2 loctaions ...

Pour un seuil de 30 minutes
 244 locations seront bloquées et 1368 loctaions seront possibles


,Seuil (min),Locations bloquées,% Loc bloquées,Locations possibles,% Loc possibles,% Retards réels couverts
0,30,244,15.14,1368,84.86,35.37
1,60,358,22.21,1254,77.79,53.36
2,90,518,32.13,1094,67.87,65.37
3,180,773,47.95,839,52.05,81.33
4,240,884,54.84,728,45.16,85.81


In [7]:
# Analyse meilleur Seuil : calcul des gains et coûts entre chaque seuil
gains_retards = [
    df_resultat['% Retards réels couverts'].iloc[i+1] - df_resultat['% Retards réels couverts'].iloc[i]
    for i in range(len(df_resultat)-1)
]
couts_locations = [
    df_resultat['% Loc bloquées'].iloc[i+1] - df_resultat['% Loc bloquées'].iloc[i]
    for i in range(len(df_resultat)-1)
]

# Calcul du ratio Gain/Coût
ratios = [
    gains_retards[i] / coûts if coûts != 0 else float('inf')
    for i, coûts in enumerate(couts_locations)
]

# Affichage des ratios pour chaque transition
print("Ratios Gain/Coût pour chaque transition entre les seuils :")
for i, (gain, cout, ratio) in enumerate(zip(gains_retards, couts_locations, ratios)):
    seuil_depart = df_resultat['Seuil (min)'].iloc[i]
    seuil_arrivee = df_resultat['Seuil (min)'].iloc[i+1]
    print(f"Transition de {seuil_depart} min à {seuil_arrivee} min :")
    print(f"  Gain en % Retards couverts = {gain:.2f}")
    print(f"  Augmentation en % Loc bloquées = {cout:.2f}")
    print(f"  Ratio Gain/Coût = {ratio:.2f}\n")

print("Seuil optimal (60 min) : Le ratio Gain/Coût est maximal à ce seuil (2.54), indiquant que chaque point supplémentaire de locations\n bloquées permet de couvrir 2.54 points de retards. Ce seuil représente donc le meilleur compromis entre efficacité et coût opérationnel.")    

# Visualisation 
fig = px.line(df_resultat, x='Seuil (min)', y=['% Retards réels couverts', '% Loc bloquées'],
              title='Compromis entre % Retards réels couverts et % Loc bloquées')
        
# Ajout d'une ligne verticale pour le seuil de 60 minutes
fig.add_vline(x=60, line_width=2, line_dash="dash", line_color="green", annotation_text="Seuil 60 min")

fig.show()

Ratios Gain/Coût pour chaque transition entre les seuils :
Transition de 30 min à 60 min :
  Gain en % Retards couverts = 17.99
  Augmentation en % Loc bloquées = 7.07
  Ratio Gain/Coût = 2.54

Transition de 60 min à 90 min :
  Gain en % Retards couverts = 12.01
  Augmentation en % Loc bloquées = 9.92
  Ratio Gain/Coût = 1.21

Transition de 90 min à 180 min :
  Gain en % Retards couverts = 15.96
  Augmentation en % Loc bloquées = 15.82
  Ratio Gain/Coût = 1.01

Transition de 180 min à 240 min :
  Gain en % Retards couverts = 4.48
  Augmentation en % Loc bloquées = 6.89
  Ratio Gain/Coût = 0.65

Seuil optimal (60 min) : Le ratio Gain/Coût est maximal à ce seuil (2.54), indiquant que chaque point supplémentaire de locations
 bloquées permet de couvrir 2.54 points de retards. Ce seuil représente donc le meilleur compromis entre efficacité et coût opérationnel.
